# A* Algo

In [1]:
import numpy as np
n = 10
matrix = []
block_unblock_matrix = []

np_array = np.random.randint(0, 100, n*n).reshape(n, n)
matrix = np.asmatrix(np_array)

matrix

matrix([[29, 93, 15, 57, 29, 95, 20,  8, 63, 17],
        [97, 34, 86, 73, 61, 92, 16, 22, 18, 40],
        [88, 16, 30, 69, 18, 85, 70, 72, 90, 96],
        [34,  2,  8, 70, 59, 97, 23, 48, 18, 63],
        [55, 42, 70, 87,  6, 29, 67,  7, 79, 71],
        [ 1, 98, 64, 85, 67, 59, 74, 30, 55, 55],
        [52, 86, 95, 24, 99, 28, 87, 23,  1, 51],
        [ 3, 77, 88, 92, 63, 16, 81, 89, 98, 11],
        [ 8, 69, 88, 77, 64, 55, 69, 11, 73, 45],
        [ 4, 11, 83, 21, 86,  7, 47, 62, 47,  4]])

In [2]:
np_array = np.random.randint(0, 2, n*n).reshape(n, n)
block_unblock_matrix = np.asmatrix(np_array)

block_unblock_matrix

matrix([[0, 0, 0, 1, 0, 1, 1, 0, 1, 1],
        [1, 1, 1, 1, 0, 0, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
        [1, 0, 0, 0, 0, 1, 0, 1, 0, 1],
        [0, 0, 0, 1, 0, 1, 1, 1, 0, 0],
        [1, 1, 0, 0, 1, 1, 0, 1, 1, 0],
        [0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
        [1, 1, 1, 0, 1, 0, 0, 0, 0, 0]])

In [3]:
# [[1 for j in range(n)] for i in range(n)]

# hofn_matrix = list(np.array(np.arange(100).reshape((10,10))))
# # np.fliplr(hofn_matrix)

def compute_hofn(n):
    k = 0
    hofn = []
    one_row = []
    for j in range(n):
        one_row.append(k)
        k += 1

    hofn.append(one_row)

    for i in range(n-1):
        one_row = [x+1 for x in one_row]
        hofn.append(one_row)

    hofn_matrix = np.array([np.array(xi) for xi in hofn])
    hofn_matrix = np.flip(hofn_matrix)
    return hofn_matrix

g(n) matrix

In [4]:
def compute_gofn(n):
    k = 0
    gofn = []
    one_row = []
    for j in range(n):
        one_row.append(k)
        k += 1
    gofn.append(one_row)
    for i in range(n-1):
        one_row = [x+1 for x in one_row]
        gofn.append(one_row)

    gofn_matrix = np.array([np.array(xi) for xi in gofn])
    return gofn_matrix

In [5]:
def get_gofn(gofn_matrix, row, col):
    return gofn_matrix[row][col]

In [6]:
def get_hofn(hofn_matrix, row, col):
    return hofn_matrix[row][col]

In [7]:
def get_children(matrix, row, column, n):
    children = []
    if row == 0 and column == 0:
        children.append((row+1,column))
        children.append((row, column+1))
    elif row == n-1 and column == n-1:
        print("goal node doesn't have children")
    elif row == n-1:
        children.append((row, column+1))
    elif column == n-1:
        children.append((row+1, column))
    else:
        children.append((row+1, column))
        children.append((row, column+1))
    return children 

In [8]:
def sort(open_list, gofn_matrix, hofn_matrix):
    sort_dict = {}
    for node in open_list:
        row = node[0]
        col = node[1]
        gofn = gofn_matrix.item(row,col)
        hofn = hofn_matrix.item(row,col)
        print("node", node, " - gofn: ", gofn,  " - hofn: ", hofn)
#         fofn = gofn + hofn
#         print("fofn: ", fofn)
        sort_dict[node] = hofn
        sort_dict = dict(sorted(sort_dict.items(), key=lambda item: item[1]))
    return sort_dict

In [9]:
visited_list = []
open_list = []

start = (0,0)
goal = (n-1, n-1)

open_list.append(start)
# print(open_list)
gofn_matrix = compute_gofn(n)
# print("gofn_matrix", gofn_matrix)
hofn_matrix = compute_hofn(n)
# print("hofn_matrix", hofn_matrix)

while open_list:
    current_node = open_list.pop()
    print("current_node:", current_node)
    if block_unblock_matrix.item(current_node) == 0:
        visited_list.append(current_node)
        children = get_children(block_unblock_matrix, current_node[0], current_node[1], n)
        print("children", children)
        open_list.extend(children)
        sort_dict = sort(open_list, gofn_matrix, hofn_matrix)
        print("open_list", open_list)
        print("sort_dict", sort_dict)
        print("---------------------\n")
    else:
        print("path blocked. Cannot traverse to goal node")
        break

current_node: (0, 0)
children [(1, 0), (0, 1)]
node (1, 0)  - gofn:  1  - hofn:  17
node (0, 1)  - gofn:  1  - hofn:  17
open_list [(1, 0), (0, 1)]
sort_dict {(1, 0): 17, (0, 1): 17}
---------------------

current_node: (0, 1)
children [(1, 1), (0, 2)]
node (1, 0)  - gofn:  1  - hofn:  17
node (1, 1)  - gofn:  2  - hofn:  16
node (0, 2)  - gofn:  2  - hofn:  16
open_list [(1, 0), (1, 1), (0, 2)]
sort_dict {(1, 1): 16, (0, 2): 16, (1, 0): 17}
---------------------

current_node: (0, 2)
children [(1, 2), (0, 3)]
node (1, 0)  - gofn:  1  - hofn:  17
node (1, 1)  - gofn:  2  - hofn:  16
node (1, 2)  - gofn:  3  - hofn:  15
node (0, 3)  - gofn:  3  - hofn:  15
open_list [(1, 0), (1, 1), (1, 2), (0, 3)]
sort_dict {(1, 2): 15, (0, 3): 15, (1, 1): 16, (1, 0): 17}
---------------------

current_node: (0, 3)
path blocked. Cannot traverse to goal node
